# Import MALT

In [1]:
from malt import Dataset
from malt import Lightcurve
from malt import interpolator 
from malt import feature_extraction 
import os

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


#  The MALT lightcurve object

In [2]:
lightcurve = Lightcurve('3C120_test', obj_type = None, interpolate = True,
                 interp_func = interpolator.get_gp, ini_t = 'rand', obs_time = 8/24,
                 sample_size = 100)

## Atributes

#### lightcurve.time
#### lightcurve.flux
#### lightcurve.flux_err
#### lightcurve.ra_dec
#### lightcurve.type
#### lightcurve.filename
#### lightcurve.interp_flux
#### lightcurve.features

## Extract features

In [ ]:
lightcurve.extract_features(feat_ex_method= feature_extraction.get_wavelet_feature )

# The MALT dataset object

In [2]:
root = 'formated_data/'
allfiles = []
for path, subdirs, files in os.walk(root):
    #subdirs[:] = [d for d in subdirs if d not in ['GBI']]
    for filename in files:
        f = os.path.join(path, filename)
        allfiles.append(f)

## Initialise MALT dataset object with parameters from config file

In [3]:
dataset = Dataset(configFile='MALT.config')

## Import files into dataset object 

In [4]:
dataset.populate(allfiles)

## Check types of objects in the dataset 

In [5]:
dataset.types()

AGN         :    17
Flare-Star  :     3
GRB         :     4
Kilonova    :     1
Magnetar    :     1
Nova        :     8
SN          :    13
TDE         :     2
--------------------
Total       :    49


In [6]:
dataset.types(show_aug_num=True)

AGN         :  1700
Flare-Star  :   300
GRB         :   400
Kilonova    :   100
Magnetar    :   100
Nova        :   800
SN          :  1300
TDE         :   200
--------------------
Total       :  4900


## Interpolate all Lightcurves in dataset

In [7]:
dataset.interpolate()

## Extract features from all Lightcurves in dataset

In [8]:
dataset.extract_features()

## Train a ML algorithm on dataset

In [9]:
dataset.train()


Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   17.0s finished


GridSearchCV took 17.54 seconds for 40 candidate parameter settings.


## Testing new lightcurve

In [6]:
new_lightcurve = Lightcurve('3C120_test')
new_lightcurve.type

'None'

## Predict class using trained dataset

In [10]:
prob, label = dataset.predict(new_lightcurve)
print(prob,label)

0.9876543209876543 AGN


In [11]:
prob,label = dataset.predict(new_lightcurve,show_prob=True)

AGN         : 98.77%
Flare-Star  : 1.23%
GRB         : 0.00%
Kilonova    : 0.00%
Magnetar    : 0.00%
Nova        : 0.00%
SN          : 0.00%
TDE         : 0.00%


## Adding new lightcurve to dataset

In [12]:
new_agn = Lightcurve('3C120_test',obj_type='AGN')
new_agn.type

'AGN'

In [13]:
dataset.add(new_agn)


Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   13.3s finished


GridSearchCV took 13.87 seconds for 40 candidate parameter settings.


In [14]:
dataset.types()

AGN         :  18
Flare-Star  :   3
GRB         :   4
Kilonova    :   1
Magnetar    :   1
Nova        :   8
SN          :  13
TDE         :   2
------------------
Total       :  50


# Saving Dataset and Loading from saved file

In [10]:
dataset.save('Malt_demo')

In [7]:
reloaded_dataset = Dataset.load_from_save('Malt_demo.save')
prob,label = reloaded_dataset.predict(new_lightcurve,show_prob=True)

AGN         : 94.29%
Flare-Star  : 2.86%
GRB         : 0.00%
Kilonova    : 0.00%
Magnetar    : 0.00%
Nova        : 0.00%
SN          : 2.86%
TDE         : 0.00%


## Everything is logged

In [16]:
cat lightcurve.log

2019-05-29 12:21:37 - lightcurve_logger - ERROR - An error occured trying to read the file.
Traceback (most recent call last):
  File "/home/kimeel/Documents/SKA-fellowship/MALT/lightcurve.py", line 13, in __init__
    t,flux,flux_err = self.loadfile(filename)
  File "/home/kimeel/Documents/SKA-fellowship/MALT/lightcurve.py", line 59, in loadfile
    data = np.loadtxt(filename)
  File "/home/kimeel/Documents/SKA-fellowship/MALT/malt_env/lib/python3.6/site-packages/numpy/lib/npyio.py", line 962, in loadtxt
    fh = np.lib._datasource.open(fname, 'rt', encoding=encoding)
  File "/home/kimeel/Documents/SKA-fellowship/MALT/malt_env/lib/python3.6/site-packages/numpy/lib/_datasource.py", line 266, in open
    return ds.open(path, mode, encoding=encoding, newline=newline)
  File "/home/kimeel/Documents/SKA-fellowship/MALT/malt_env/lib/python3.6/site-packages/numpy/lib/_datasource.py", line 624, in open
    raise IOError("%s not found." % path)
OSError: 0458-02 not found.
2019-05-29 12:21:37 -

2019-07-22 21:47:48 - lightcurve_logger - INFO - Func get_gp() used saved gp: ./saved/gps/NGC4278.txt.gpsave
2019-07-22 21:47:48 - lightcurve_logger - INFO - Func get_gp() used saved gp: ./saved/gps/2223-052.gpsave
2019-07-22 21:47:48 - lightcurve_logger - INFO - Func get_gp() used saved gp: ./saved/gps/0458-020.gpsave
2019-07-22 21:47:48 - lightcurve_logger - INFO - Func get_gp() used saved gp: ./saved/gps/SN1988z.txt.gpsave
2019-07-22 21:47:48 - lightcurve_logger - INFO - Func get_gp() used saved gp: ./saved/gps/V1500Cyg.txt.gpsave
2019-07-22 21:47:48 - lightcurve_logger - INFO - Func get_gp() used saved gp: ./saved/gps/V1723Aql.txt.gpsave
2019-07-22 21:47:48 - lightcurve_logger - INFO - Func get_gp() used saved gp: ./saved/gps/SN2008iz.gpsave
2019-07-22 21:47:48 - lightcurve_logger - INFO - Func get_gp() used saved gp: ./saved/gps/V1974Cyg.txt.gpsave
2019-07-22 21:47:48 - lightcurve_logger - INFO - Func get_gp() used saved gp: ./saved/gps/RSOph.txt.gpsave
2019-07-22 21:47:48 - light

In [10]:
'.save' not in 'Malt_demo'

True

In [ ]:
boolean('False')